This notebook will be a tutorial in how to calculate volume, as a difference between consecutive DEMs.
A rough overview of the steps to be followed is given below-
1. Sort DEMs in chronological order. 
2. Inspect DEMs and Mosaics to check for distortions. 
3. Create Vector Mask to remove Areas with distortions. 
4. Mask original DEM with created vector mask. 
5. Use grass's `r.fillnulls` command to create the gaps left by the masked distortions. 
6. Obtain set of error free DEMs.
7. Using this set, obtain delta DEMs between consecutive DEMs, or as per requirement. 
8. Do summation over obtained Delta DEM and multiply by area of a pixel, as per scale to obtain volume. 


*Note- Steps with a * \* *indicates that there are algorithms/variables to be tuned*

In [1]:
import gdal
import cv2
from subprocess import call
import numpy as np
import matplotlib.pyplot as plt
#Custom Written Scripts
try:
    import dem_proj as dp
    import dem_filtering as df
    import dem_perf_check as dpc
    from raster_chunks import GeoChunks as gc 
    
except ImportError:
    print("Import Error. Check if dem_filtering.py is present in PYTHONPATH")
    print("PYTHONPATH = ")
    call(["echo", "$PYTHONPATH"])

# Bit of formatting to change default inline code style:
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")

Setting up PATH variables. 

We will continue to set up PATH variables, as and when necessary.



In [2]:
PATH_PREFIX = "/home/madhavm/vimana/workdir/rough/cs_volume_2/"
t1_dem = PATH_PREFIX + "T_F_t1_dem.tif"
t2_dem = PATH_PREFIX + "T_F_t2_dem.tif"
#T3 DEM is yet to be processed.
#t3_dem = PATH_PREFIX + "T_F_t3_dem.tif"
t4_dem = PATH_PREFIX + "T_F_t4_dem.tif"
t5_dem = PATH_PREFIX + "T_F_t5_dem.tif"
t6_dem = PATH_PREFIX + "T_F_t6_dem.tif"
pixel_area = 0.027606668620293 ** 2

Next, we ensure that all the DEMs have the same extent and resolution,  
to ensure that the Delta DEM calculated does not throw any unexpected errors. 

In [3]:
dpc.match_shape_DEM(referenceDEM=t1_dem,terrainDEM=t2_dem)
#dpc.match_shape_DEM(referenceDEM=t1_dem,terrainDEM=t3_dem)
dpc.match_shape_DEM(referenceDEM=t1_dem,terrainDEM=t4_dem)
dpc.match_shape_DEM(referenceDEM=t1_dem,terrainDEM=t5_dem)
dpc.match_shape_DEM(referenceDEM=t1_dem,terrainDEM=t6_dem)

Now, all DEMs have the same resolution and extent, so we can proceed  
to calculate the Delta DEMs. Before calculation, we first set the  
PATH variables.


In [3]:
t1_t2_delta = PATH_PREFIX + "D_t1_t2.tif"
#t1_t3_delta = PATH_PREFIX + "D_t1_t3.tif"
t1_t4_delta = PATH_PREFIX + "D_t1_t4.tif"
t1_t5_delta = PATH_PREFIX + "D_t1_t5.tif"
t1_t6_delta = PATH_PREFIX + "D_t1_t6.tif"
#Delta DEMs between consecutive sessions

#t2_t3_delta = PATH_PREFIX + "D_t2_t3.tif"
#t3_t4_delta = PATH_PREFIX + "D_t3_t4.tif"
t4_t5_delta = PATH_PREFIX + "D_t4_t5.tif"
t5_t6_delta = PATH_PREFIX + "D_t5_t6.tif"

Now, for Delta DEM processing.

In [5]:
dpc.create_delta_DEM(referenceDEM=t1_dem,terrainDEM=t2_dem,deltaDEM=t1_t2_delta)
#dpc.create_delta_DEM(referenceDEM=t1_dem,terrainDEM=t3_dem,deltaDEM=t1_t3_delta)
dpc.create_delta_DEM(referenceDEM=t1_dem,terrainDEM=t4_dem,deltaDEM=t1_t4_delta)
dpc.create_delta_DEM(referenceDEM=t1_dem,terrainDEM=t5_dem,deltaDEM=t1_t5_delta)
dpc.create_delta_DEM(referenceDEM=t1_dem,terrainDEM=t6_dem,deltaDEM=t1_t6_delta)

#Delta DEMs between consecutive sessions

#dpc.create_delta_DEM(referenceDEM=t2_dem,terrainDEM=t3_dem,deltaDEM=t2_t3_delta)
#dpc.create_delta_DEM(referenceDEM=t3_dem,terrainDEM=t4_dem,deltaDEM=t3_t4_delta)
dpc.create_delta_DEM(referenceDEM=t4_dem,terrainDEM=t5_dem,deltaDEM=t4_t5_delta)
dpc.create_delta_DEM(referenceDEM=t5_dem,terrainDEM=t6_dem,deltaDEM=t5_t6_delta)

In [4]:
mask_area = PATH_PREFIX + "excav_area_WGS84.shp"
masked_t1_t2_delta = PATH_PREFIX + "M_D_1_t2.tif"
#masked_t1_t3_delta = PATH_PREFIX + "M_D_t1_t3.tif"
masked_t1_t4_delta = PATH_PREFIX + "M_D_t1_t4.tif"
masked_t1_t5_delta = PATH_PREFIX + "M_D_t1_t5.tif"
masked_t1_t6_delta = PATH_PREFIX + "M_D_t1_t6.tif"
#Delta DEMs between consecutive sessions

#masked_t2_t3_delta = PATH_PREFIX + "M_D_t2_t3.tif"
#masked_t3_t4_delta = PATH_PREFIX + "M_D_t3_t4.tif"
masked_t4_t5_delta = PATH_PREFIX + "M_D_t4_t5.tif"
masked_t5_t6_delta = PATH_PREFIX + "M_D_t5_t6.tif"

In [10]:
#dpc.applyMask(baseDEM=input_DEM, maskDEM=masked_DEM,maskPoly=input_trim)
dpc.apply_mask_poly(baseDEM=t1_t2_delta, maskDEM=masked_t1_t2_delta,maskPoly=mask_area)
#dpc.apply_mask_poly(baseDEM=t1_t3_delta, maskDEM=masked_t1_t3_delta,maskPoly=mask_area)
dpc.apply_mask_poly(baseDEM=t1_t4_delta, maskDEM=masked_t1_t4_delta,maskPoly=mask_area)
dpc.apply_mask_poly(baseDEM=t1_t5_delta, maskDEM=masked_t1_t5_delta,maskPoly=mask_area)
dpc.apply_mask_poly(baseDEM=t1_t6_delta, maskDEM=masked_t1_t6_delta,maskPoly=mask_area)
#Delta DEMs between consecutive sessions

#dpc.apply_mask_poly(baseDEM=t2_t3_delta, maskDEM=masked_t2_t3_delta,maskPoly=mask_area)
#dpc.apply_mask_poly(baseDEM=t3_t4_delta, maskDEM=masked_t3_t4_delta,maskPoly=mask_area)
dpc.apply_mask_poly(baseDEM=t4_t5_delta, maskDEM=masked_t4_t5_delta,maskPoly=mask_area)
dpc.apply_mask_poly(baseDEM=t5_t6_delta, maskDEM=masked_t5_t6_delta,maskPoly=mask_area)


Mask Operation Completed.
Output file present at /home/madhavm/vimana/rough/cs_volume/volume_proc/M_D_1_t2.tif
Mask Operation Completed.
Output file present at /home/madhavm/vimana/rough/cs_volume/volume_proc/M_D_t1_t4.tif
Mask Operation Completed.
Output file present at /home/madhavm/vimana/rough/cs_volume/volume_proc/M_D_t1_t5.tif
Mask Operation Completed.
Output file present at /home/madhavm/vimana/rough/cs_volume/volume_proc/M_D_t1_t6.tif
Mask Operation Completed.
Output file present at /home/madhavm/vimana/rough/cs_volume/volume_proc/M_D_t4_t5.tif
Mask Operation Completed.
Output file present at /home/madhavm/vimana/rough/cs_volume/volume_proc/M_D_t5_t6.tif


In [5]:
fixed_t1_t2_delta = PATH_PREFIX + "F_D_1_t2.tif"
#fixed_t1_t3_delta = PATH_PREFIX + "F_D_t1_t3.tif"
fixed_t1_t4_delta = PATH_PREFIX + "F_D_t1_t4.tif"
fixed_t1_t5_delta = PATH_PREFIX + "F_D_t1_t5.tif"
fixed_t1_t6_delta = PATH_PREFIX + "F_D_t1_t6.tif"
#Delta DEMs between consecutive sessions

#fixed_t2_t3_delta = PATH_PREFIX + "F_D_t2_t3.tif"
#fixed_t3_t4_delta = PATH_PREFIX + "F_D_t3_t4.tif"
fixed_t4_t5_delta = PATH_PREFIX + "F_D_t4_t5.tif"
fixed_t5_t6_delta = PATH_PREFIX + "F_D_t5_t6.tif"

In [15]:
df.maskDEM(mask_input=masked_t1_t2_delta, dem_input=masked_t1_t2_delta,file_output=fixed_t1_t2_delta,val_threshold=1000)
#df.maskDEM(mask_input=masked_t1_t3_delta, dem_input=masked_t1_t3_delta,file_output=fixed_t1_t3_delta,val_threshold=1000)
df.maskDEM(mask_input=masked_t1_t4_delta, dem_input=masked_t1_t4_delta,file_output=fixed_t1_t4_delta,val_threshold=1000)
df.maskDEM(mask_input=masked_t1_t5_delta, dem_input=masked_t1_t5_delta,file_output=fixed_t1_t5_delta,val_threshold=1000)
df.maskDEM(mask_input=masked_t1_t6_delta, dem_input=masked_t1_t6_delta,file_output=fixed_t1_t6_delta,val_threshold=1000)
#Delta DEMs between consecutive sessions

#df.maskDEM(mask_input=masked_t2_t3_delta, dem_input=masked_t2_t3_delta,file_output=fixed_t2_t3_delta,val_threshold=1000)
#df.maskDEM(mask_input=masked_t3_t4_delta, dem_input=masked_t3_t4_delta,file_output=fixed_t3_t4_delta,val_threshold=1000)
df.maskDEM(mask_input=masked_t4_t5_delta, dem_input=masked_t4_t5_delta,file_output=fixed_t4_t5_delta,val_threshold=1000)
df.maskDEM(mask_input=masked_t5_t6_delta, dem_input=masked_t5_t6_delta,file_output=fixed_t5_t6_delta,val_threshold=1000)

True

In [8]:
vol_t1_t2 = df.sumDEM(fixed_t1_t2_delta) * pixel_area
#vol_t1_t3 = df.sumDEM(fixed_t1_t3_delta) * pixel_area
vol_t1_t4 = df.sumDEM(fixed_t1_t4_delta) * pixel_area
vol_t1_t5 = df.sumDEM(fixed_t1_t5_delta) * pixel_area
vol_t1_t6 = df.sumDEM(fixed_t1_t6_delta) * pixel_area

#vol_t2_t3 = df.sumDEM(fixed_t2_t3_delta) * pixel_area
#vol_t3_t4 = df.sumDEM(fixed_t3_t4_delta) * pixel_area
vol_t4_t5 = df.sumDEM(fixed_t4_t5_delta) * pixel_area
vol_t5_t6 = df.sumDEM(fixed_t5_t6_delta) * pixel_area

print("All volumes are in cubic metres - \n")
print("Volume excavated between session 1 and session 2 - " + str(vol_t1_t2))
#print("Volume excavated between session 1 and session 3 - " + str(vol_t1_t3))
print("Volume excavated between session 1 and session 4 - " + str(vol_t1_t4))
print("Volume excavated between session 1 and session 5 - " + str(vol_t1_t5))
print("Volume excavated between session 1 and session 6 - " + str(vol_t1_t6))

print("\nDelta Volumes from consecutive sessions..\n")
#print("Volume excavated between session 2 and session 3 - " + str(vol_t2_t3))
#print("Volume excavated between session 3 and session 4 - " + str(vol_t3_t4))
print("Volume excavated between session 4 and session 5 - " + str(vol_t4_t5))
print("Volume excavated between session 5 and session 6 - " + str(vol_t5_t6))


All volumes are in cubic metres - 

Volume excavated between session 1 and session 2 - 1893.14768114
Volume excavated between session 1 and session 4 - 3626.72097849
Volume excavated between session 1 and session 5 - 16282.8064202
Volume excavated between session 1 and session 6 - 19085.2615399

Delta Volumes from consecutive sessions..

Volume excavated between session 4 and session 5 - 12656.0430483
Volume excavated between session 5 and session 6 - 2802.48688173


Approaching the problem with a different approach,  

In [3]:
utm_t1_dem = PATH_PREFIX + "UTM/t1_dem.tif"
utm_t2_dem = PATH_PREFIX + "UTM/t2_dem.tif"
#utm_t3_dem = PATH_PREFIX + "UTM/t3_dem.tif"
utm_t4_dem = PATH_PREFIX + "UTM/t4_dem.tif"
utm_t5_dem = PATH_PREFIX + "UTM/t5_dem.tif"
utm_t6_dem = PATH_PREFIX + "UTM/t6_dem.tif"


In [4]:
t1 = gc(utm_t1_dem)
t2 = gc(utm_t2_dem)
#t3 = gc(utm_t3_dem)
t4 = gc(utm_t4_dem)
t5 = gc(utm_t5_dem)
t6 = gc(utm_t6_dem)

pa_t1 = t1.geo_trans[1] ** 2
pa_t2 = t2.geo_trans[1] ** 2
#pa_t3 = t3.geo_trans[1] ** 2
pa_t4 = t4.geo_trans[1] ** 2
pa_t5 = t5.geo_trans[1] ** 2
pa_t6 = t6.geo_trans[1] ** 2


In [5]:
m2_vol_t1 = df.sumDEM(utm_t1_dem) * pa_t1
m2_vol_t2 = df.sumDEM(utm_t2_dem) * pa_t2
#m2_vol_t3 = df.sumDEM(utm_t3_dem) * pa_t3
m2_vol_t4 = df.sumDEM(utm_t4_dem) * pa_t4
m2_vol_t5 = df.sumDEM(utm_t5_dem) * pa_t5
m2_vol_t6 = df.sumDEM(utm_t6_dem) * pa_t6

print("All volumes are in cubic metres - \n")
print("Volume excavated between session 1 and session 2 - " + str(m2_vol_t1 - m2_vol_t2))
#print("Volume excavated between session 1 and session 3 - " + str(m2_vol_t1 - m2_vol_t3))
print("Volume excavated between session 1 and session 4 - " + str(m2_vol_t1 - m2_vol_t4))
print("Volume excavated between session 1 and session 5 - " + str(m2_vol_t1 - m2_vol_t5))
print("Volume excavated between session 1 and session 6 - " + str(m2_vol_t1 - m2_vol_t6))

print("\nDelta Volumes from consecutive sessions..\n")
#print("Volume excavated between session 2 and session 3 - " + str(m2_vol_t2 - m2_vol_t3))
#print("Volume excavated between session 3 and session 4 - " + str(m2_vol_t3 - m2_vol_t4))
print("Volume excavated between session 4 and session 5 - " + str(m2_vol_t4 - m2_vol_t5))
print("Volume excavated between session 5 and session 6 - " + str(m2_vol_t5 - m2_vol_t6))


All volumes are in cubic metres - 

Volume excavated between session 1 and session 2 - 1894.49674766
Volume excavated between session 1 and session 4 - 3630.29215387
Volume excavated between session 1 and session 5 - 16302.7828132
Volume excavated between session 1 and session 6 - 19091.6169702

Delta Volumes from consecutive sessions..

Volume excavated between session 4 and session 5 - 12672.4906593
Volume excavated between session 5 and session 6 - 2788.83415697
